In [1]:
!pip install pandas
import pandas as pd
dataset = pd.read_csv('try1.csv',sep=';')
dataset.head()

,TIME,TRANSACTION_ID,AMOUNT,CLIENT_ID,BANK_ID,CITY,OPERATION_TYPE,FRAUD_FLAG
0,2023-05-01,525471381920,1560,BD308B1E6247DAD7E7FAC425AB3CADA464C26F2A0B47EB...,6,VSEVOLOZHSK,Снятие через банкоматы партнеров,0
1,2023-05-01,525471383640,1560,BD308B1E6247DAD7E7FAC425AB3CADA464C26F2A0B47EB...,6,VSEVOLOZHSK,Снятие через банкоматы партнеров,0
2,2023-05-01,525487382296,290,2E021B8D64532F869A14167A6075996B903D52ED66B3AD...,6,OBNINSK,Снятие через банкоматы партнеров,0
3,2023-05-01,525364464247,310,B27401D40DC566402D47BA342DDD6A1457E91F3D211D0B...,3,CHITA G,Пополнение через банкоматы партнеров,0
4,2023-05-01,525513617413,140,B566176D8952A9B1AA4A25D1574AE1CA9EA57C2AB4F29C...,2,DALNEGORSK,Снятие через чужие банкоматы,0


In [2]:
dataset['TIME'] = pd.to_datetime(dataset['TIME'])

In [4]:
dataset['YEAR_MONTH'] = dataset['TIME'].dt.to_period('M')

In [6]:
monthly_dataset = dataset.groupby(['YEAR_MONTH', 'BANK_ID', 'OPERATION_TYPE'])['AMOUNT'].sum().reset_index()

In [7]:
monthly_dataset = monthly_dataset.sort_values(by=['YEAR_MONTH', 'BANK_ID', 'OPERATION_TYPE'])

In [8]:
monthly_dataset['PREV_AMOUNT'] = monthly_dataset.groupby(['BANK_ID', 'OPERATION_TYPE'])['AMOUNT'].shift(1)
monthly_dataset['GROWTH'] = monthly_dataset['AMOUNT'] - monthly_dataset['PREV_AMOUNT']
monthly_dataset['PERCENT_GROWTH'] = (monthly_dataset['GROWTH'] / monthly_dataset['PREV_AMOUNT']) * 100

In [9]:
monthly_dataset = monthly_dataset.dropna()

In [10]:
max_percent_growth = monthly_dataset.loc[monthly_dataset['PERCENT_GROWTH'].idxmax()]
max_nominal_growth = monthly_dataset.loc[monthly_dataset['GROWTH'].idxmax()]

In [12]:
print("Максимальный процентный рост:")
print(f"Банк: {max_percent_growth['BANK_ID']}, Тип операции: {max_percent_growth['OPERATION_TYPE']}, "
      f"Месяц: {max_percent_growth['YEAR_MONTH']}, Процентный рост: {max_percent_growth['PERCENT_GROWTH']:.2f}%")

print("Максимальный номинальный рост:")
print(f"Банк: {max_nominal_growth['BANK_ID']}, Тип операции: {max_nominal_growth['OPERATION_TYPE']}, "
      f"Месяц: {max_nominal_growth['YEAR_MONTH']}, Номинальный рост: {max_nominal_growth['GROWTH']:.2f}")

Максимальный процентный рост:
Банк: 35, Тип операции: Снятие через чужие банкоматы, Месяц: 2023-07, Процентный рост: 510.84%
Максимальный номинальный рост:
Банк: 3, Тип операции: Пополнение через банкоматы партнеров, Месяц: 2023-07, Номинальный рост: 20759850.00
